In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from Utils import extract_symbol_timeframe

path = " " # Format filename = {symbol}USDT_timeframe_data.csv, example: "BTCUSDT_1h_from_1_Jan_2020.csv"
df = pd.read_csv(path)

symbol, timeframe = extract_symbol_timeframe(path)
indicator_name = "simple_strategy"

# === ПАРАМЕТРЫ ===
impulse_threshold = 1     # % падения
target_profit_pct = 0.2      # take profit
stop_loss_pct = 99      # stop loss
window = 20                # свечей в анализе

# === ЗАГРУЗКА ДАННЫХ ===
# Убедись, что df содержит: 'open_time', 'high', 'low', 'close'
df["open_time"] = pd.to_datetime(df["open_time"], utc=True)

highs = df["high"].to_numpy(dtype=np.float64)
lows = df["low"].to_numpy(dtype=np.float64)
closes = df["close"].to_numpy(dtype=np.float64)
times = df["open_time"].values.astype("datetime64[ns]")

# === ВЫЧИСЛЕНИЕ ИМПУЛЬСОВ ===
rolling_max = pd.Series(closes).rolling(window=window).max().to_numpy()
rolling_min = pd.Series(closes).rolling(window=window).min().to_numpy()
drawdowns = (rolling_min / rolling_max - 1) * 100
impulse_mask = drawdowns <= -impulse_threshold

# === БЭКТЕСТ С ПРОГРЕССБАРОМ ===
in_trade = False
results = []

i = 0
n = len(closes)
pbar = tqdm(total=n, desc="🚀 Backtesting")

while i < n:
    if not in_trade and impulse_mask[i]:
        entry_price = closes[i]
        entry_time = times[i]
        take_profit = entry_price * (1 + target_profit_pct / 100)
        stop_loss = entry_price * (1 - stop_loss_pct / 100)
        in_trade = True

        j = i + 1
        while j < n:
            if highs[j] >= take_profit:
                exit_price = take_profit
                exit_time = times[j]
                exit_type = "take_profit"
                break
            elif lows[j] <= stop_loss:
                exit_price = stop_loss
                exit_time = times[j]
                exit_type = "stop_loss"
                break
            j += 1
        else:
            print("⚠️ Неисполненная сделка:", times[i])
            break

        duration = (exit_time - entry_time) / np.timedelta64(1, "m")
        profit_pct = (exit_price / entry_price - 1) * 100

        results.append({
            "entry_time": entry_time,
            "entry_price": entry_price,
            "exit_time": exit_time,
            "exit_price": exit_price,
            "exit_type": exit_type,
            "duration": duration,
            "profit_pct": profit_pct,
            "stop_loss": stop_loss,
            "stopped_out": exit_type == "stop_loss"
        })

        in_trade = False
        pbar.update(j - i)
        i = j
    else:
        i += 1
        pbar.update(1)

pbar.close()

# === СТАТИСТИКА ===
result_df = pd.DataFrame(results)

print("📈 Сделок:", len(result_df))
print("✅ Прибыльных:", (result_df["exit_type"] == "take_profit").sum())
print("💥 Стопов:", result_df["stopped_out"].sum())
print("🕒 Среднее время (мин):", result_df["duration"].mean())

balance = 10
for p in result_df["profit_pct"]:
    balance *= (1 + p / 100)
print(f"💰 Финальный капитал: ${balance:.2f}")

print(f"Криптовалюта: {symbol}\nТаймфрейм: {timeframe}\nСтратегия: {indicator_name}\nИмпульс падения: {impulse_threshold}\nОкно: {window}\nПрофит: {target_profit_pct}%")

# Примеры
result_df.tail(3)
